In [36]:
import os
import zipfile
import glob
import numpy as np
import pandas as pd
import torch
from scipy.interpolate import interp1d
from skimage.feature import local_binary_pattern
import cv2
from tqdm import tqdm
from dataset import EmotionDataset 
import mediapipe as mp

In [37]:
def check_sample_availability(sample_dir):
    # check video data
    video_files = [f for f in os.listdir(sample_dir) 
                   if f.endswith('.mp4') or f.endswith('.avi') or f.endswith('-Video.zip')]
    if not video_files:
        return False

    # check annotation files（Valence, Arousal, Liking）
    annotation_types = ['Valence', 'Arousal', 'Liking']
    for atype in annotation_types:
        annotation_zip_files = [f for f in os.listdir(sample_dir) if f'-{atype}.zip' in f]
        if not annotation_zip_files:
            return False

    return True


In [46]:
def process_landmarks(sample_dir):
    video_files = [f for f in os.listdir(sample_dir) 
                   if f.endswith('.mp4') or f.endswith('.avi') or f.endswith('-Video.zip')]
    if not video_files:
        return None, None
    
    video_file = os.path.join(sample_dir, video_files[0])
    
    if video_file.endswith('-Video.zip'):
        video_extract_dir = os.path.join(sample_dir, 'Videos')
        if not os.path.exists(video_extract_dir):
            os.makedirs(video_extract_dir, exist_ok=True)
            with zipfile.ZipFile(video_file, 'r') as zip_ref:
                zip_ref.extractall(video_extract_dir)
        video_files = [f for f in os.listdir(video_extract_dir) 
                       if f.endswith('.mp4') or f.endswith('.avi')]
        if not video_files:
            return None, None
        video_file = os.path.join(video_extract_dir, video_files[0])
    
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False,
                                      max_num_faces=1,
                                      refine_landmarks=True,
                                      min_detection_confidence=0.5,
                                      min_tracking_confidence=0.5)
    
    cap = cv2.VideoCapture(video_file)
    landmarks_seq = []
    frame_indices = []
    frame_idx = 0
    image_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    image_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)
        
        if results.multi_face_landmarks:
            face_landmarks = results.multi_face_landmarks[0]
            landmarks = []
            for lm in face_landmarks.landmark:
                landmarks.append(lm.x * image_width)  
                landmarks.append(lm.y * image_height)
            landmarks_seq.append(landmarks)
            frame_indices.append(frame_idx)
        
        frame_idx += 1
    
    cap.release()
    face_mesh.close()
    
    if not landmarks_seq:
        return None, None  
    
    landmarks_seq = np.array(landmarks_seq) 
    frame_indices = np.array(frame_indices) 
    return landmarks_seq, frame_indices

In [39]:
def extract_lbp_features(image, num_points=8, radius=1, grid_x=7, grid_y=7):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp_image = local_binary_pattern(gray, num_points, radius, method='uniform')
    height, width = lbp_image.shape
    grid_height = height // grid_y
    grid_width = width // grid_x
    lbp_features = []
    for i in range(grid_y):
        for j in range(grid_x):
            grid = lbp_image[i*grid_height:(i+1)*grid_height, j*grid_width:(j+1)*grid_width]
            hist, _ = np.histogram(grid.ravel(), bins=np.arange(0, num_points+3), range=(0, num_points+2))
            hist = hist.astype('float')
            hist /= (hist.sum() + 1e-7)
            lbp_features.extend(hist)
    return np.array(lbp_features)

In [40]:
def process_lbp_sequence(video_path, frame_indices):
    cap = cv2.VideoCapture(video_path)
    lbp_sequence = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices_set = set(frame_indices)
    current_frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if current_frame_idx in frame_indices_set:
            lbp_feature = extract_lbp_features(frame)
            lbp_sequence.append(lbp_feature)
        current_frame_idx += 1
    cap.release()
    lbp_sequence = np.array(lbp_sequence)  # Shape: (num_frames_with_landmarks, feature_dim)
    return lbp_sequence

In [41]:
def process_annotations(sample_dir, annotation_type):
    # Find the annotation zip file
    annotation_zip_files = [f for f in os.listdir(sample_dir) if f'-{annotation_type}.zip' in f]
    if not annotation_zip_files:
        return None
    annotation_zip = os.path.join(sample_dir, annotation_zip_files[0])

    # Extract annotations
    annotations_extract_dir = os.path.join(sample_dir, f'{annotation_type}')
    if not os.path.exists(annotations_extract_dir):
        os.makedirs(annotations_extract_dir, exist_ok=True)
        with zipfile.ZipFile(annotation_zip, 'r') as zip_ref:
            zip_ref.extractall(annotations_extract_dir)

    # Read annotations from CSV files (video annotations only)
    csv_files = glob.glob(os.path.join(annotations_extract_dir, f'*_{annotation_type}_V*.csv'))
    annotations_list = []
    for csv_file in csv_files:
        df = pd.read_csv(csv_file, header=None)
        times = df.iloc[:, 0].values  # Time column
        values = df.iloc[:, 1].values  # Annotation values
        annotations_list.append((times, values))
    if not annotations_list:
        return None

    # Average annotations from different annotators
    # First, create a time axis common to all annotators
    min_time = max(annot[0][0] for annot in annotations_list)
    max_time = min(annot[0][-1] for annot in annotations_list)
    if max_time <= min_time:
        return None  # invalid time range
    common_times = np.linspace(min_time, max_time, num=round((max_time - min_time) * 66))  # Assuming 66 Hz sampling rate

    # Interpolate annotations to the common time axis
    interpolated_values = []
    for times, values in annotations_list:
        interp_func = interp1d(times, values, kind='linear', fill_value='extrapolate')
        interpolated_values.append(interp_func(common_times))
    # Average the interpolated values
    annotations_seq = np.mean(interpolated_values, axis=0)  # Shape: (seq_len,)
    # Return both common times and averaged annotations
    return common_times, annotations_seq

In [42]:
def align_sequences(landmarks_seq, lbp_seq, frame_indices, frame_timestamps, annotations_times, annotations_seq):
    # frame_indices: indices of frames with valid landmarks and LBP features
    # frame_timestamps: timestamps corresponding to frame_indices
    # annotations_times: timestamps of annotations
    # annotations_seq: annotation values

    # Interpolate annotations to match frame timestamps
    interp_func = interp1d(annotations_times, annotations_seq, kind='linear', fill_value='extrapolate')
    labels_seq = interp_func(frame_timestamps)

    # Ensure all sequences have the same length
    seq_len = min(len(landmarks_seq), len(lbp_seq), len(labels_seq))
    landmarks_seq = landmarks_seq[:seq_len]
    lbp_seq = lbp_seq[:seq_len]
    labels_seq = labels_seq[:seq_len]

    return landmarks_seq, lbp_seq, labels_seq

In [43]:
def save_processed_data(sample_id, landmarks_seq, lbp_seq, labels_seq, seq_len):
    processed_dir = os.path.join('processed_data', f'sample_{sample_id}')
    os.makedirs(processed_dir, exist_ok=True)
    np.save(os.path.join(processed_dir, 'landmarks.npy'), landmarks_seq)
    np.save(os.path.join(processed_dir, 'lbp.npy'), lbp_seq)
    np.save(os.path.join(processed_dir, 'Valence.npy'), labels_seq[:, 0])
    np.save(os.path.join(processed_dir, 'Arousal.npy'), labels_seq[:, 1])
    np.save(os.path.join(processed_dir, 'Liking.npy'), labels_seq[:, 2])
    np.save(os.path.join(processed_dir, 'seq_len.npy'), np.array([seq_len]))

In [44]:
def process_all_samples(dataset_dir, desired_seq_len=None):
    sample_dirs = [os.path.join(dataset_dir, d) for d in os.listdir(dataset_dir)
                   if os.path.isdir(os.path.join(dataset_dir, d))]
    processed_count = 0
    for sample_dir in tqdm(sample_dirs, desc='Processing samples'):
        sample_id = os.path.basename(sample_dir)
        # check the video
        if not check_sample_availability(sample_dir):
            print(f"Skipping {sample_id}: Missing required files.")
            continue
        try:
            # process Landmarks
            landmarks_seq, frame_indices = process_landmarks(sample_dir)
            if landmarks_seq is None or len(landmarks_seq) == 0:
                print(f"Skipping {sample_id}: No valid landmarks.")
                continue

            video_files = [f for f in os.listdir(sample_dir) 
                           if f.endswith('.mp4') or f.endswith('.avi') or f.endswith('-Video.zip')]
            if video_files:
                video_file = os.path.join(sample_dir, video_files[0])
                if video_file.endswith('-Video.zip'):
                    video_extract_dir = os.path.join(sample_dir, 'Videos')
                    if not os.path.exists(video_extract_dir):
                        os.makedirs(video_extract_dir, exist_ok=True)
                        with zipfile.ZipFile(video_file, 'r') as zip_ref:
                            zip_ref.extractall(video_extract_dir)
                    video_files = [f for f in os.listdir(video_extract_dir) 
                                   if f.endswith('.mp4') or f.endswith('.avi')]
                    if not video_files:
                        print(f"Skipping {sample_id}: No video file found after extraction.")
                        continue
                    video_file = os.path.join(video_extract_dir, video_files[0])
                
                cap = cv2.VideoCapture(video_file)
                fps = cap.get(cv2.CAP_PROP_FPS)
                cap.release()
                if fps <= 0:
                    print(f"Skipping {sample_id}: Invalid FPS ({fps}).")
                    continue
            else:
                print(f"Skipping {sample_id}: No video file found.")
                continue

            frame_timestamps = frame_indices / fps

            # process LBP
            lbp_seq = process_lbp_sequence(video_file, frame_indices)
            if lbp_seq is None or len(lbp_seq) == 0:
                print(f"Skipping {sample_id}: No valid LBP features.")
                continue

            # process annotations
            annotations = {}
            for atype in ['Valence', 'Arousal', 'Liking']:
                annotations_data = process_annotations(sample_dir, atype)
                if annotations_data is None:
                    print(f"Skipping {sample_id}: Missing or invalid {atype} annotations.")
                    break  
                annotations[atype] = annotations_data
            else:
                # align sequence for each emotion dimension
                landmarks_seq_aligned, lbp_seq_aligned, valence_seq = align_sequences(
                    landmarks_seq, lbp_seq, frame_indices, frame_timestamps,
                    annotations['Valence'][0], annotations['Valence'][1]
                )
                _, _, arousal_seq = align_sequences(
                    landmarks_seq_aligned, lbp_seq_aligned, frame_indices, frame_timestamps,
                    annotations['Arousal'][0], annotations['Arousal'][1]
                )
                _, _, liking_seq = align_sequences(
                    landmarks_seq_aligned, lbp_seq_aligned, frame_indices, frame_timestamps,
                    annotations['Liking'][0], annotations['Liking'][1]
                )
                # Merge tags
                labels_seq = np.stack([valence_seq, arousal_seq, liking_seq], axis=1)  # Shape: (seq_len, 3)

                # Process sequence length
                seq_len = len(labels_seq)
                if desired_seq_len is not None:
                    if seq_len < desired_seq_len:
                        # Skip samples with insufficient sequence length
                        print(f"Skipping {sample_id}: Sequence too short ({seq_len} frames).")
                        continue
                    else:
                        # Randomly intercept a subsequence
                        start_idx = np.random.randint(0, seq_len - desired_seq_len + 1)
                        end_idx = start_idx + desired_seq_len
                        landmarks_seq_aligned = landmarks_seq_aligned[start_idx:end_idx]
                        lbp_seq_aligned = lbp_seq_aligned[start_idx:end_idx]
                        labels_seq = labels_seq[start_idx:end_idx]
                        seq_len = desired_seq_len

                # save processed data
                save_processed_data(sample_id, landmarks_seq_aligned, lbp_seq_aligned, labels_seq, seq_len, sample_id)
                processed_count += 1
        except Exception as e:
            print(f"Error processing {sample_id}: {e}")
            continue
    print(f"Processed {processed_count} samples.")

In [47]:
dataset_dir ='SEWA_DB'  
desired_seq_len = 50 

process_all_samples(dataset_dir, desired_seq_len)

Processing samples:   0%| | 0/2630 [00:00<?,I0000 00:00:1733371486.784688 67237106 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
W0000 00:00:1733371486.787080 67442803 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733371486.795650 67442799 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Processing samples:   0%| | 1/2630 [00:08<6:

Skipping SSD_C3_S092_P183_VC1_003651_004631: Missing or invalid Valence annotations.
Skipping SVH_C3_S085_P170_VC1_000901_001561: Missing required files.
Skipping C1_S020_P039_AD2: Missing required files.
Skipping SVL_C2_S062_P123_VC1_000489_001586: Missing required files.


I0000 00:00:1733371495.615611 67237106 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
W0000 00:00:1733371495.617466 67443009 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733371495.622200 67443014 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Processing samples:   0%| | 5/2630 [00:17<2:

Skipping SSD_C1_S001_P001_VC1_004201_005201: Missing or invalid Valence annotations.
Skipping SVL_C2_S048_P096_VC1_000837_001711: Missing required files.


I0000 00:00:1733371504.345046 67237106 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
W0000 00:00:1733371504.346495 67443360 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733371504.352360 67443362 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Processing samples:   0%| | 7/2630 [00:38<4:

...

Skipping C1_S194_P388_AD4: Missing or invalid Valence annotations.
Skipping C5_S156_P311_AD1: Missing required files.
Skipping C4_S096_P192_AD3: Missing required files.
Skipping C1_S017_P033_VC1: Missing required files.
Skipping C4_S103_P206_AD3: Missing required files.
Skipping C3_S072_P143_AD2: Missing required files.


I0000 00:00:1733374837.324671 67237106 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
W0000 00:00:1733374837.328577 67519971 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733374837.346526 67519971 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Processing samples:  39%|▍| 1029/2630 [56:15

Skipping C2_S057_P113_AD4: Missing or invalid Valence annotations.
Skipping C4_S123_P246_AD2: Missing required files.
Skipping C6_S174_P347_AD2: Missing required files.
Skipping C3_S068_P135_VC1: Missing required files.
Skipping SVH_C2_S062_P123_VC1_003722_004499: Missing required files.
Skipping C3_S079_P157_AD3: Missing required files.


I0000 00:00:1733374862.766375 67237106 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
W0000 00:00:1733374862.768312 67520584 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733374862.774774 67520585 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Processing samples:  39%|▍| 1034/2630 [56:24


KeyboardInterrupt: 

In [ ]:
from torch.utils.data import DataLoader

processed_data_dir = 'processed_data'  
dataset = EmotionDataset(processed_data_dir)


batch_size = 8
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# test DataLoader
for batch in dataloader:
    landmarks_seq, lbp_seq, labels_seq = batch
    print("Landmarks shape:", landmarks_seq.shape)
    print("LBP shape:", lbp_seq.shape)
    print("Labels shape:", labels_seq.shape)
    break  
